## HEAD

#### HEAD 01 - toggle user settings

In [1]:
## determine whether to cache data from some time consuming tasks
settings = {
    'test_mode': False, ## when true, only a small of data is collected
    'collect_data': False, ## toggles twitter api pulls in PULL01-03
    'rebuild_word_data': True, ## rebuild vs cache load for word_data MUNG02
    'rebuild_tweet_words': True, ## rebuild vs cache load tweet_words MUNG03
    'rebuild_user_token': True ## rebuild vs cache load user_token_tally MUNG04
    }

#### HEAD02 - load libraries

In [2]:
##########==========##########==========##########==========##########==========
import tweepy
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
from time import sleep
from os.path import exists
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA

#### HEAD03 - load data files

In [3]:
## read in roster of handles
user_data = pd.read_excel("A_Input/twitter_handles.xlsx")

## drop all but a handful of cases if in test mode
if settings['test_mode']:
    user_data = user_data.sample(100, random_state = 5542)

## read in twitter credentials; initialize api connection+
twitter_credentials = pd.read_csv('../api_keys/twitter.csv').set_index('item')
twitter_credentials = tweepy.OAuth1UserHandler(
    consumer_key = twitter_credentials.loc['API Key', 'string'],
    consumer_secret = twitter_credentials.loc['API Key Secret', 'string'],
    access_token = twitter_credentials.loc['Access Token', 'string'],
   access_token_secret = twitter_credentials.loc['Access Token Secret', 'string']
    )
api = tweepy.API(twitter_credentials)

/Users/s8/opt/anaconda3/envs/py310/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


#### HEAD04 - create build or cache decision function

In [4]:
## build switching function to execute code or cache results
def build_or_cache(address, function, build_bool):
    if build_bool or not exists(address):
        x = function()
        x.to_csv(address, index = False)
        return x
    else:
        return pd.read_csv(address)            

## HAND – Gather Twitter handles for test accounts

#### HAND01 - extract handles from roster URLs

In [5]:
## extract handles from roster urls
user_data['handle'] = user_data.url.str.replace('https://twitter.com/', '',
            regex = False).str.strip().str.lower()

## PULL - Pull Twitter data from the API

#### PULL01 - query API for each roster handle's user_timeline data

In [6]:
## extract tweet data from api object
def refine_tweet_data(x):
    tweet_data = list()
    for i in range(0, len(x)):
        tweet_data.append({
            'tweet_id': x[i].id, 'created_at': x[i].created_at, 'lang': x[i].lang,
            'full_text': x[i].full_text,
            'screen_name': x[i].author.screen_name,
            'verified' : x[i].author.verified
        })
    return pd.DataFrame(tweet_data)

## define function to pull user tweet data and apply function to extract tweet data
def pull_tweet_data(handles = user_data.handle, a = api):
    tweet_data = list()
    for i in handles:
        try:
            user_tweets = a.user_timeline(
                screen_name = i, count = 200, tweet_mode = 'extended', 
                exclude_replies = True, include_rts = False)
            tweet_data.append(refine_tweet_data(user_tweets))
            sleep(0.5)
        except:
            pass
    tweet_data = pd.concat(tweet_data)
    tweet_data['screen_name'] = tweet_data['screen_name'].str.lower()
    return tweet_data

## execute code
if settings['collect_data']:
    tweet_data = pull_tweet_data()

#### PULL02 - tabulate tweet statistics, divide users into train/tune/test sets

In [7]:
## combile tweet count and verification status at the user level
def enhance_user_data(td, ud = user_data):
    
    def np_size(x): return x.size
    
    ## calculate tweet summary statistics
    td_original = tweet_data
    td = td.copy()
    verified = td.groupby('screen_name').mean()
    tweets = td['screen_name'].value_counts()
    td = pd.concat([verified, tweets], axis = 1).reset_index()
    td.columns = ['handle', 'verified', 'tweets']
    
    ## merge statistics into the user_data object
    ud = pd.merge(ud, td, on = 'handle', how = 'left')
    td_original = td_original.drop(['verified'], axis = 1)
    ud = ud.drop(['url'], axis = 1).reset_index(drop = True)
    ud = ud.fillna({'tweets': 0}).astype({'tweets': int})
    
    ## divide users into train, tune, and test subsets
    ml_set = pd.Series(['train', 'tune', 'test'], name = 'ml_set').sample(
                n = ud.shape[0], replace = True, weights = [0.7, 0.15, 0.15],
                random_state = 2006)
    ud['ml_set'] = ml_set.values
    ud.loc[ud.tweets == 0, 'ml_set'] = 'exclude'
    
    return ud, td_original

## execute code
if settings['collect_data']:
    user_data, tweet_data = enhance_user_data(
        tweet_data[['screen_name', 'verified']])

#### PULL03 - save datasets to disk

In [8]:
## save user/tweet datasets to disk as csvs
if settings['collect_data']:
    user_data.to_csv('B_Process/user_data.csv', index = False)
    tweet_data.to_csv('B_Process/tweet_data.csv', index = False)
else:
    user_data = pd.read_csv('B_Process/user_data.csv')
    tweet_data = pd.read_csv('B_Process/tweet_data.csv')

## MUNG - Process Twitter data to model-ready format

#### MUNG01 - parse tweet text into tokens

In [9]:
## tokenize, remove capitalization, and remove duplicate tokens
def nlp_tokenize_tweet(x):
    x = x.lower()
    x = word_tokenize(x)
    x = list(set(x))
    return x

## execute code
tweet_data['tokens'] = tweet_data.full_text.apply(nlp_tokenize_tweet)

#### MUNG02 - create word/token level dataset

In [10]:
## create word/token level dataset and identify valid word tokens
def make_word_data(td = tweet_data):

    ## flatten token lists and count occurances
    word_data = list()
    for i in td.tokens:
        word_data += i
    word_data = pd.Series(word_data, name = 'count').value_counts()
    word_data = word_data.sort_values(ascending = False)
    word_data = pd.DataFrame(word_data)
    
    ## determine which tokens occur often enough to warrant inclusion
    word_data['valid'] = word_data['count'] > max(
        word_data['count'].quantile(0.2), 3)
    word_data['word'] = word_data.index
    
    ## determine part of speech for eligible tokens
    speech_part = word_data['word'].loc[word_data['valid']].values
    speech_part = pos_tag(speech_part)
    speech_part = [i[1][0].lower() for i in speech_part]
    word_data['pos'] = '.'
    word_data.loc[word_data['valid'], 'pos'] = speech_part
    
    ## lemmatize
    WNL = WordNetLemmatizer()
    word_data['token'] = None
    for i in word_data.word:
        if not word_data.loc[i, 'valid']: 
            break
        if word_data.loc[i, 'pos'] in 'abcdefghijklmnopqrstuvwxyz':
            try:
                word_data.loc[i, 'token'] = WNL.lemmatize(
                    word_data.loc[i, 'word'],
                    pos = word_data.loc[i, 'pos']
                )
            except:
                word_data.loc[i, 'token'] = word_data.loc[i, 'word']
        else:
            word_data.loc[i, 'valid'] = False
        
    return word_data.reset_index(drop = True)

## execute code
word_data = build_or_cache(
    address = 'B_Process/word_data.csv',
    function = make_word_data,
    build_bool = settings['rebuild_word_data']
    )

#### MUNG03 - generate a tokens x tweets link database

In [11]:
def make_tweet_token_data(td = tweet_data, wd = word_data):
    
    wd = wd.set_index('word')
    
    ## replicate tweet ids
    n = td.tokens.apply(len).values
    tweet_tokens = pd.Series(np.repeat(td.tweet_id.values, n), name = "tweet_id")
    tweet_tokens = pd.DataFrame(tweet_tokens)
    
    ## allocate words to the new dataset
    words = list()
    for i in td.tokens:
        words += i
    tweet_tokens['words'] = words
    
    ## convert words to tokens
    tweet_tokens['tokens'] = wd.loc[
        tweet_tokens.words.values, 'token'].values
    tweet_tokens = tweet_tokens.dropna()

    return tweet_tokens.reset_index(drop = True)

## execute code
tweet_words = build_or_cache(
    address = 'B_Process/tweet_words.csv',
    function = make_tweet_token_data,
    build_bool = settings['rebuild_tweet_words']
    )
tweet_data = tweet_data.drop('tokens', axis = 1)

#### MUNG04 - generate a tokens x users count; drop tokens with only one user

In [14]:
def make_user_token_matrix(td = tweet_data, tw = tweet_words, ud = user_data):
    
    ## count of number of times each user wrote each token
    tw = tw.merge(right = td[['screen_name', 'tweet_id']],
                  how = 'left', on = 'tweet_id')
    tw = tw.drop(['tweet_id', 'words'], axis = 1).groupby('screen_name')
    tw = tw.value_counts()
    tw.name = 'count'
    tw = tw.reset_index().set_index('screen_name')
    tw = tw.pivot(columns = 'tokens').fillna(0).astype(int)
    tw = tw.droplevel(axis = 1, level = 0)
    
    ## remove tokens that fewer than 3 or more than 80 percent of users use
    valid_usage = (tw > 0).astype(int).sum().values
    valid_usage = (valid_usage > 2
                  ) & (valid_usage < int(tw.shape[0] * 0.8))
    tw = tw.loc[:, valid_usage]
    
    ## standardize matrix as words per 1,000 tweets
    denom = pd.DataFrame({'handle': tw.index}).merge(
        ud[['handle', 'tweets']],
        how = 'left', on = 'handle'
        ).set_index('handle').squeeze().fillna(1)
    denom.loc[denom < 1] = 1
    tw = (tw.divide(denom, axis = 0) * 1000).astype(int)
    
    return tw.reset_index()


## execute code
user_token_matrix = build_or_cache(
    address = 'B_Process/user_token_matrix.csv',
    function = make_user_token_matrix,
    build_bool = settings['rebuild_user_token']
    ).set_index('screen_name')

KeyError: "None of ['screen_name'] are in the columns"

In [15]:
print('TODO: fix missing screen_name issue')

,tweet_id,words,tokens
0,1552727930408734720,the,the
1,1552727930408734720,african,african
2,1552727930408734720,https,http
3,1552727930408734720,photos,photo
4,1552727930408734720,@,@
...,...,...,...
2960509,1506387676655759365,family,family
2960510,1506387676655759365,from,from
2960511,1506387676655759365,alaskans,alaskans
2960512,1506387676655759365,to,to


## TRAI – Train models and tune hyperparameters

##### reminders
##### - standardize data
##### - write a two-stage meta-function
##### - do automatic feature selection for regression?

#### TRAI00 - Unpack train, tune, and test datasets

In [ ]:
## unpack train, tune, test datasets
def split_xy_data(ml_cat):
    i = user_data.loc[user_data.ml_set == ml_cat, 'handle'].values
    
    x = user_token_matrix.loc[i, :]
    
    y1 = user_data.set_index('handle').loc[i, 'group'] == 'USA House'
    y1 = y1.astype(int)
    
    y2 = user_data.set_index('handle').loc[i, 'party'] == 'Republican'
    y2 = y2.astype(int)
    
    return x, y1, y2

## execute code
train_x, train_y1, train_y2 = split_xy_data('train')
tune_x, tune_y1, tune_y2 = split_xy_data('tune')
test_x, test_y1, test_y2 = split_xy_data('test')

In [ ]:
## delete objects to clear up memory
del tweet_words, tweet_data, user_token_matrix, word_data

#### TRAI01 - generate PCA simplification of features matrix

In [ ]:
## fit pca model in order to simplify and enhance feature matrix
model_pca = PCA().fit(train_x)

## generate elbow curve statistics for number of components
def pca_elbow(mod = model_pca, fig_name = None):
    
    ## calculate statistics
    quality = mod.singular_values_
    quality = (np.cumsum(quality) / np.sum(quality)).round(3)
    n = list(range(1, len(quality) + 1))
    elbow_stats = pd.DataFrame({'n': n, 'quality': quality})
    
    ## output figure
    print('TODO: finish building section')
    
    ## find elbow
    
    return elbow_stats

pca_elbow().to_csv('elbow.csv')

## choose number of components
print('TODO: finishing building PCA section')

## generate pca transformations of all feature matrices
train_x_pca = model_pca.transform(train_x)
tune_x_pca  = model_pca.transform(tune_x)
test_x_pca  = model_pca.transform(test_x)

#### TRAI02 - [random]

#### TRAI03 - [logistic regression (features, PCA)]

#### TRAI04 - [Naive bayes (features, pca)]

#### TRAI05 - [random forest (features, pca)]

#### TRAI06 - [adaboost (features, pca)]

In [16]:
user_data

,group,party,member,state,handle,verified,tweets,ml_set
0,USA House,Democratic,"Adams, Alma",North Carolina,repadams,1.0,87,train
1,USA House,Republican,"Aderholt, Robert",Alabama,robert_aderholt,1.0,154,test
2,USA House,Democratic,"Aguilar, Pete",California,reppeteaguilar,1.0,133,train
3,USA House,Republican,"Allen, Rick",Georgia,reprickallen,1.0,150,train
4,USA House,Democratic,"Allred, Colin",Texas,repcolinallred,1.0,170,tune
...,...,...,...,...,...,...,...,...
791,CAN House,Conservative,"Williamson, John",New Brunswick,johnwilliamson_,1.0,127,tune
792,CAN House,Liberal,"Yip, Jean",Ontario,jeanyip3,1.0,170,train
793,CAN House,Liberal,"Zahid, Salma",Ontario,salmazahid15,1.0,135,train
794,CAN House,NDP,"Zarrillo, Bonita",British Columbia,bonitazarrillo,1.0,32,train


In [17]:
user_data.ml_set.value_counts()

train      551
tune       133
test       101
exclude     11
Name: ml_set, dtype: int64